# Transforma em DataFrame e limpa

In [18]:
import pandas as pd

# Lê excel e transforma em DataFrame
vendas_df = pd.read_excel('../../../data/silver/2025_11_treated.xlsx')

In [19]:
# Cria tupla de sufixos de colunas necessários
sufixos_necessários = tuple(['Vendas', 'Anúncios', 'Publicidade'])

# Remove colunas desnecessárias
for col in vendas_df.columns:

    if not col.endswith(sufixos_necessários):
        vendas_df = vendas_df.drop(col, axis=1)

In [20]:
# Filtra apenas colunas de vendas concluídas
vendas_df = vendas_df[
    # Valor total da venda precisa ser maior que 0
    (vendas_df['Total_(BRL)_Vendas'] != 0) &
    # É preciso que haja o título do anúncio, a ausência indica que a venda não foi concluída
    (vendas_df['Título_do_anúncio_Anúncios'].notna()) &
    (vendas_df['Título_do_anúncio_Anúncios'].str.strip() != '')
]